In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Dropout

# 데이터셋 로드
train = pd.read_csv('combined_dataset5.csv')

# 데이터셋을 훈련 및 테스트 세트로 분할
train_data, test_data = train_test_split(train, test_size=0.2, random_state=42)

# 입력 데이터 준비
input_texts = train_data['가사'].astype(str).tolist()
target_texts = train_data['요약'].astype(str).tolist()

# 입력 및 타겟 텍스트 토큰화 및 시퀀스 패딩
tokenizer_input = Tokenizer()
tokenizer_input.fit_on_texts(input_texts)
input_sequences = tokenizer_input.texts_to_sequences(input_texts)
input_sequences = pad_sequences(input_sequences)

tokenizer_target = Tokenizer()
tokenizer_target.fit_on_texts(target_texts)
target_sequences = tokenizer_target.texts_to_sequences(target_texts)
target_sequences = pad_sequences(target_sequences, padding='post')

# 모델 아키텍처 정의
embedding_size = 128
latent_dim = 256

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(len(tokenizer_input.word_index) + 1, embedding_size)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(len(tokenizer_target.word_index) + 1, embedding_size)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(tokenizer_target.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델 컴파일
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit([input_sequences, target_sequences[:, :-1]], target_sequences[:, 1:], epochs=20, batch_size=64, validation_split=0.2)

# 테스트 세트에서 모델 평가 및 예측
test_input_texts = test_data['가사'].astype(str).tolist()
test_target_texts = test_data['요약'].astype(str).tolist()

test_input_sequences = tokenizer_input.texts_to_sequences(test_input_texts)
test_input_sequences = pad_sequences(test_input_sequences)

test_target_sequences = tokenizer_target.texts_to_sequences(test_target_texts)
test_target_sequences = pad_sequences(test_target_sequences, padding='post')

# 테스트 세트에서 예측 생성
predictions = model.predict([test_input_sequences, test_target_sequences[:, :-1]])
predicted_sequences = []
for prediction in predictions.argmax(axis=-1):
    predicted_sequence = [tokenizer_target.index_word.get(idx, '') for idx in prediction]
    predicted_sequences.append(' '.join(predicted_sequence))

# 정확도 및 손실 표시
evaluation = model.evaluate([test_input_sequences, test_target_sequences[:, :-1]], test_target_sequences[:, 1:])
print("테스트 정확도:", evaluation[1])
print("테스트 손실:", evaluation[0])

# 실제 및 예측된 요약 표시
for i in range(len(test_data)):
    print("\n실제 요약:", test_target_texts[i])
    print("예측된 요약:", predicted_sequences[i])

Epoch 1/20
6/6 [==============================] - 42s 6s/step - loss: 7.3589 - accuracy: 0.4784 - val_loss: 7.0214 - val_accuracy: 0.5972
Epoch 2/20
6/6 [==============================] - 31s 5s/step - loss: 5.3541 - accuracy: 0.6008 - val_loss: 3.4268 - val_accuracy: 0.5972
Epoch 3/20
6/6 [==============================] - 32s 5s/step - loss: 3.1871 - accuracy: 0.6008 - val_loss: 3.0904 - val_accuracy: 0.5972
Epoch 4/20
6/6 [==============================] - 34s 6s/step - loss: 2.9593 - accuracy: 0.6012 - val_loss: 3.0644 - val_accuracy: 0.5993
Epoch 5/20
6/6 [==============================] - 36s 6s/step - loss: 2.8853 - accuracy: 0.6023 - val_loss: 3.0975 - val_accuracy: 0.5993
Epoch 6/20
6/6 [==============================] - 35s 6s/step - loss: 2.8367 - accuracy: 0.6029 - val_loss: 3.1040 - val_accuracy: 0.6003
Epoch 7/20
6/6 [==============================] - 35s 6s/step - loss: 2.8132 - accuracy: 0.6042 - val_loss: 3.1555 - val_accuracy: 0.6003
Epoch 8/20
6/6 [==================